In [ ]:
import csv
import json
import requests

In [ ]:
BASE_URL = 'http://localhost:8080'
GET_LIKES = BASE_URL + '/api/chatbot_feedback/likes'
GET_DISLIKES = BASE_URL + '/api/chatbot_feedback/dislikes'
GET_SELECTED_REPLIES = BASE_URL + '/api/chatbot_feedback/selected_replies'
POST_UPDATE_SELECTED = BASE_URL + '/api/chatbot_feedback/batch_update_selected'

In [ ]:
# like example
def print_like_list(like_list):
    for like in like_list:
        Content = like.get('Content', '')
        ReplyID = like.get('ReplyID', -1)
        Selected = like.get('Selected', False)
        UserID = like.get('UserID', -1)
        print(f'ReplyID: {ReplyID}, Selected: {Selected}, UserID: {UserID}\nContent: {Content}\n')
    print('')
    return

params = {
    'start_date': '2024-11-01',
    'end_date': '2024-11-30',
    'page': 1,
}

try:
    response = requests.get(GET_LIKES, params=params)
    response.raise_for_status()
    data = response.json()
    like_list = data.get('data', [])
    max_page = data.get('maxpage', 1)
    print_like_list(like_list)
    print(f'Max page: {max_page}')
except Exception as e:
    print(f'Error: {e}')


In [ ]:
# dislike example
def print_dislike_list(dislike_list):
    for dislike in dislike_list:
        Content = dislike.get('Content', '')
        Feedback = dislike.get('Feedback', '')
        ReplyID = dislike.get('ReplyID', -1)
        Selected = dislike.get('Selected', False)
        UserID = dislike.get('UserID', -1)
        print(f'ReplyID: {ReplyID}, Selected: {Selected}, UserID: {UserID}\nContent: {Content}\nFeedback: {Feedback}\n')
    print('')
    return

params = {
    'start_date': '2024-11-01',
    'end_date': '2024-11-30',
    'page': 1,
}

try:
    response = requests.get(GET_DISLIKES, params=params)
    response.raise_for_status()
    data = response.json()
    dislike_list = data.get('data', [])
    max_page = data.get('maxpage', 1)
    print_dislike_list(dislike_list)
    print(f'Max page: {max_page}')
except Exception as e:
    print(f'Error: {e}')


In [ ]:
# selected replies example
def save_selected_replies_json(selected_replies, filename):
    """
    Save selected replies to a JSON file.
    """
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(selected_replies, f, ensure_ascii=False, indent=4)
    return

def save_selected_replies_csv(selected_replies, filename):
    """
    Save selected replies to a CSV file.
    """
    with open(filename, 'w', encoding='utf-8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=["text_input", "output"])
        writer.writeheader()
        writer.writerows(selected_replies)
    return

params = {
    'start_date': '2024-11-01',
    'end_date': '2024-11-30',
    'page': 1,
}

replies = []
while True:
    try:
        response = requests.get(GET_SELECTED_REPLIES, params=params)
        response.raise_for_status()
        data = response.json()
        selected_replies = data.get('data', [])
        max_page = data.get('maxpage', 1)
        replies.extend(selected_replies)
        if params['page'] >= max_page:
            break
        params['page'] += 1
    except Exception as e:
        print(f'Error: {e}')
        break

save_selected_replies_json(replies, 'selected_replies.json')
# save_selected_replies_csv(replies, 'selected_replies.csv')


For model-tuning, refer:

- https://ai.google.dev/gemini-api/docs/model-tuning/tutorial?lang=python
- https://github.com/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/model-tuning/python.ipynb

In [ ]:
# update selected example
input_data = like_list
input_data[0]['Content'] = 'new content'
input_data[1]['Selected'] = False

try:
    response = requests.post(POST_UPDATE_SELECTED, json=input_data)
    response.raise_for_status()
    data = response.json()
    print(data)
except Exception as e:
    print(f'Error: {e}')
    